# Подготовка ноутбука 

## Пробрасываем magic methods

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

## Imports

In [ ]:
import pandas as pd 
import numpy as np 
from utils.s3 import s3_download_file, get_s3_client, s3_list_files
from utils.zip import extract_zip_advanced, move_and_remove_files
from dotenv import load_dotenv
import os 
from pathlib import Path
from warnings import filterwarnings 
import torch 
import torchvision
from IPython.display import display
import shutil

## Нужные переменные 

In [ ]:
ROOT_DIR = Path('../../')
load_dotenv()
filterwarnings(action='ignore')

# Загрузка датафрейма

## Пробрасываем креды

In [ ]:
get_s3_client()

## Смотрим, что есть в хранилище 

In [ ]:
s3_list_files()

## Загружаем из s3

In [ ]:
s3_download_file(file_s3_src='data/Датасет.zip', 
                 file_local_dst=ROOT_DIR / 'data/raw_data/data.zip', 
                 bucket_name='s3-dvc',)

## Извлекаем из zip

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/data.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/data',
                     remove_after_extract=True,)

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/data/Датасет/Любые объекты.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/data/any_objects',
                     remove_after_extract=True,)

# После загрузки мы перенесем фотки из ïεíδÑ «íΩÑ¬Γδ в корневую any_objects
source_dir = ROOT_DIR / 'data/raw_data/data/any_objects/ïεíδÑ «íΩÑ¬Γδ'
destination_dir = ROOT_DIR / 'data/raw_data/data/any_objects/'

move_and_remove_files(source_dir=source_dir, 
                      destination_dir=destination_dir,
                      remove_after_move=True)


In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/data/Датасет/Метаданные.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/data/metadata',
                     remove_after_extract=True,)

# Переносим в корневую metadata
destination_dir = ROOT_DIR / 'data/raw_data/data/metadata'

move_and_remove_files(source_dir=ROOT_DIR / 'data/raw_data/data/metadata/ïεíδÑ «íΩÑ¬Γδ/è¿¡ß', 
                      destination_dir=destination_dir,)
move_and_remove_files(source_dir=ROOT_DIR / 'data/raw_data/data/metadata/ïεíδÑ «íΩÑ¬Γδ/êìæ', 
                      destination_dir=destination_dir,)

shutil.move(src=ROOT_DIR / 'data/raw_data/data/metadata/îÑΓáñá¡¡δÑ/êìæ.zip',
            dst=ROOT_DIR / 'data/raw_data/data/metadata')
shutil.move(src=ROOT_DIR / 'data/raw_data/data/metadata/îÑΓáñá¡¡δÑ/è¿¡ß.zip',
            dst=ROOT_DIR / 'data/raw_data/data/metadata')
# Удаляем ненужную îÑΓáñá¡¡δÑ
shutil.rmtree(ROOT_DIR / 'data/raw_data/data/metadata/îÑΓáñá¡¡δÑ')

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/data/Датасет/Пример снимков БПЛА с ббох.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/data/bpla',
                     remove_after_extract=True,)

# После загрузки мы перенесем фотки из Åα¿¼Ñα ß¡¿¼¬«ó üÅïÇ ß íí«σ в корневую bpla
source_dir = ROOT_DIR / 'data/raw_data/data/bpla/Åα¿¼Ñα ß¡¿¼¬«ó üÅïÇ ß íí«σ'
destination_dir = ROOT_DIR / 'data/raw_data/data/bpla'

move_and_remove_files(source_dir=source_dir, 
                      destination_dir=destination_dir,
                      remove_after_move=True)

In [ ]:
shutil.rmtree(ROOT_DIR / 'data/raw_data/data/Датасет')

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/data/metadata/êìæ.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/data/metadata/êìæ',
                     remove_after_extract=True,)
# Переименовываем в нормальные названия 
os.rename(src=ROOT_DIR / 'data/raw_data/data/metadata/êìæ',
          dst=ROOT_DIR / 'data/raw_data/data/metadata/INC')

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/data/metadata/è¿¡ß.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/data/metadata/è¿¡ß',
                     remove_after_extract=True,)
os.rename(src=ROOT_DIR / 'data/raw_data/data/metadata/è¿¡ß',
          dst=ROOT_DIR / 'data/raw_data/data/metadata/Kins')